In [1]:
#code from https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Levenshtein_distance#Python
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

In [2]:
levenshtein("I am Ge Gao, a cs student", "Ge Gao is a cs major student")

14

# NLTK Wordnet

In [3]:
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import WordNetError
from nltk import word_tokenize, pos_tag   ##implementation of similarity() from http://nlpforhackers.io/wordnet-sentence-similarity/
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
def pathsim(ss1, ss2):
    if ss1.path_similarity(ss2) is None:
        return 0
    return ss1.path_similarity(ss2)
def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    NoneType = type(None)
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence1nv = []
    sentence2 = pos_tag(word_tokenize(sentence2))
    sentence2nv = []
    # Get the synsets for the tagged words
    synsets1 = []
    count1 = 0
    count2=0
    tup = ()
    synsetnone1 = []
    for tagged_word in sentence1:
        if type(tagged_to_synset(*tagged_word))== NoneType and tagged_word[0][0].isupper() and penn_to_wn(tagged_word[1]) == 'n':
            count1 += 1
            tup = (tagged_word[0], tagged_word[1])
            sentence1nv.append(tup)
            synsetnone1.append(tup)
            continue
        if penn_to_wn(tagged_word[1]) == 'n':
            count1 += 1
            tup = (tagged_word[0], tagged_word[1])
            sentence1nv.append(tup)
    synsets2 = []
    tup = ()
    synsetnone2 = []
    for tagged_word in sentence2:
        if type(tagged_to_synset(*tagged_word))== NoneType and tagged_word[0][0].isupper() and penn_to_wn(tagged_word[1]) == 'n':
            count2+=1
            tup = (tagged_word[0], tagged_word[1])
            sentence2nv.append(tup)
            synsetnone2.append(tup)
            continue
        if penn_to_wn(tagged_word[1]) == 'n':
            count2+=1
            tup = (tagged_word[0], tagged_word[1])
            sentence2nv.append(tup)
    synsets1 = [tagged_to_synset(tagged_word, tag) for tagged_word, tag in sentence1nv]
    synsets2 = [tagged_to_synset(tagged_word, tag) for tagged_word, tag in sentence2nv]
    
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if not type(ss) == NoneType]
    synsets2 = [ss for ss in synsets2 if not type(ss) == NoneType]
    score = 0.0
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        if not synsets2 == []:
            best_score = max([pathsim(synset, ss) for ss in synsets2])
        # Check that the similarity could have been computed
            if best_score is not None:
                score += best_score
    for synset in synsets2:
        # Get the similarity value of the most similar word in the other sentence
        if not synsets1 == []:
            best_score = max([pathsim(synset, ss) for ss in synsets1])
        # Check that the similarity could have been computed
            if best_score is not None:
                score += best_score
    for synsetnone in synsetnone2:
        if synsetnone in synsetnone1:
            score += 1
    for synsetnone in synsetnone1:
        if synsetnone in synsetnone2:
            score += 1
    # Average the values
    score /= (count1 + count2)
    return score

0.07083333333333333
0.05357142857142857
0.35541310541310533
0.11077626077626078
0.06993589743589743
0.0
0.08810062486533073
0.10714285714285714
0.13333333333333333
0.08035714285714285
0.0
0.07179487179487179
0.06352941176470588
0.0735042735042735
0.09444444444444444
0.07075320512820513
0.08624708624708625
0.048484848484848485
0.14431818181818182
0.0
0.1527777777777778
0.0
 
0.17833333333333332
0.13133903133903135
0.06277056277056278
0.38033395176252316
0.11798941798941798
0.0
0.09711399711399711
0.07875457875457875
0.11388888888888889
0.063003663003663
0.0
0.09023310023310023
0.10958994708994708
0.10833333333333334
0.11527777777777778
0.16111111111111112
0.10328282828282828
0.06947657605552343
0.10793650793650796
0.0
0.15936507936507938
0.0
 
0.08920940170940171
0.07766439909297052
0.12460456210456211
0.15124458874458874
0.8701298701298701
0.0
0.15665584415584416
0.07188375350140055
0.08639455782312926
0.0599031279178338
0.0
0.20334595959595958
0.10492840492840493
0.0837995337995338
0.

In [ ]:
sentences = [
    "Where can I find information about Sympa?",
    "Where is Dr. Chan's office?",
    "What is Sympa?"
    "Can I view my online courses with my PC?",
    "Can I tell other people my TRACKS pass code?",
    "Do the computer labs offer free softwares?",
    "How to install VPN",
    "Can I forword Florida Tech emails to other email addresses?",
    "How often should I check my email",
    "What kind of files can be attached to my email?",
    "How should I get emails from fitforum?",
    "What is Sympa?",
    "How do I find my Windows address?",
    "How can I activate internet for my dorm room?",
    "What is an ethernet card?",
    "What should I do if items I scan always goes to 'Junk'",
    "My computer is infected by viruses, what should I do?",
    "How can I check my final grades?",
    "What should I do if my laptop is stolen?",
    "buying hardwares for University Use",
    "How do I take a screenshot?",
    "Can I make submissions to the on-line events?",
    "Who can use cloud.fit.edu"
]
 
focus_sentences = ["How do I access my courses online?",
                   "What should I do if someone asks me about my TRACKS password?",
                   "What software is available in university computer labs?",
                   "Installing Fortinet SSL VPN Client",
                   "How do I forward emails from my Florida Tech email address to another email address?",
                   "Do I have to check my Florida Tech email address?",
                   "What is the maximum file size allowed for email attachments?",
                   "How do I subscribe to fitforum?",
                   "Information about Sympa, Florida Tech's List Server.",
                   "How do I find my MAC address?",
                   "When will my dorm room network connection be active?",
                   "What is an ethernet card?",
                   "How can I prevent items I scan to my email from going in the 'Junk' folder?",
                   "What is the best way to protect my computer from viruses, hackers, and malware?",
                   "How do I view my final grades?",
                   "What should I do if my computer is stolen?",
                   "Purchasing Computers for University Use",
                   "How do I take a screenshot?",
                   "How can I submit an on-campus event?",
                   "How do I share folders with cloud.fit.edu"]
for focus_sentence in focus_sentences:
    for sentence in sentences:
        print(sentence_similarity(focus_sentence, sentence))
    print(' ')

In [4]:
print(sentence_similarity("dogs are awesome", "cats are beautiful animals"))

0.2888888888888889


#  Entity Checking

## Question

In [22]:
def entityChecking(s):
    if "Who" in s or "who" in s:
        print("Person.")
    if "What" in s or "what" in s:
        print("Object")
    if "When" in s or "when" in s:
        print("Time")
    if "Where" in s or "where" in s:
        print("Place")
    if "How" in s or "how" in s:
        print("How")

In [23]:
entityChecking("What day is today? who are you?")

Person.
Object


## Answer

In [46]:
import os
import collections
 
ner_tags = collections.Counter()
 
corpus_root = "gmb-2.2.0"   # Make sure you set the proper path to the unzipped corpus
 
for root, dirs, files in os.walk(corpus_root):
    for filename in files:
        if filename.endswith(".tags"):
            with open(os.path.join(root, filename), 'rb') as file_handle:
                file_content = file_handle.read().decode('utf-8').strip()
                annotated_sentences = file_content.split('\n\n')   # Split sentences
                for annotated_sentence in annotated_sentences:
                    annotated_tokens = [seq for seq in annotated_sentence.split('\n') if seq]  # Split words
 
                    standard_form_tokens = []
 
                    for idx, annotated_token in enumerate(annotated_tokens):
                        annotations = annotated_token.split('\t')   # Split annotations
                        word, tag, ner = annotations[0], annotations[1], annotations[3]
 
                        ner_tags[ner] += 1
print(ner_tags)
"""
Counter({u'O': 1146068, u'geo-nam': 58388, u'org-nam': 48034, u'per-nam': 23790, u'gpe-nam': 20680, u'tim-dat': 12786, u'tim-dow': 11404, u'per-tit': 9800, u'per-fam': 8152, u'tim-yoc': 5290, u'tim-moy': 4262, u'per-giv': 2413, u'tim-clo': 891, u'art-nam': 866, u'eve-nam': 602, u'nat-nam': 300, u'tim-nam': 146, u'eve-ord': 107, u'per-ini': 60, u'org-leg': 60, u'per-ord': 38, u'tim-dom': 10, u'per-mid': 1, u'art-add': 1})
"""

Counter({'O': 1146068, 'geo-nam': 58388, 'org-nam': 48034, 'per-nam': 23790, 'gpe-nam': 20680, 'tim-dat': 12786, 'tim-dow': 11404, 'per-tit': 9800, 'per-fam': 8152, 'tim-yoc': 5290, 'tim-moy': 4262, 'per-giv': 2413, 'tim-clo': 891, 'art-nam': 866, 'eve-nam': 602, 'nat-nam': 300, 'tim-nam': 146, 'eve-ord': 107, 'org-leg': 60, 'per-ini': 60, 'per-ord': 38, 'tim-dom': 10, 'per-mid': 1, 'art-add': 1})


"\nCounter({u'O': 1146068, u'geo-nam': 58388, u'org-nam': 48034, u'per-nam': 23790, u'gpe-nam': 20680, u'tim-dat': 12786, u'tim-dow': 11404, u'per-tit': 9800, u'per-fam': 8152, u'tim-yoc': 5290, u'tim-moy': 4262, u'per-giv': 2413, u'tim-clo': 891, u'art-nam': 866, u'eve-nam': 602, u'nat-nam': 300, u'tim-nam': 146, u'eve-ord': 107, u'per-ini': 60, u'org-leg': 60, u'per-ord': 38, u'tim-dom': 10, u'per-mid': 1, u'art-add': 1})\n"

In [47]:
ner_tags = collections.Counter()
 
for root, dirs, files in os.walk(corpus_root):
    for filename in files:
        if filename.endswith(".tags"):
            with open(os.path.join(root, filename), 'rb') as file_handle:
                file_content = file_handle.read().decode('utf-8').strip()
                annotated_sentences = file_content.split('\n\n')   # Split sentences
                for annotated_sentence in annotated_sentences:
                    annotated_tokens = [seq for seq in annotated_sentence.split('\n') if seq]  # Split words
 
                    standard_form_tokens = []
 
                    for idx, annotated_token in enumerate(annotated_tokens):
                        annotations = annotated_token.split('\t')   # Split annotation
                        word, tag, ner = annotations[0], annotations[1], annotations[3]
 
                        # Get only the primary category
                        if ner != 'O':
                            ner = ner.split('-')[0]
 
                        ner_tags[ner] += 1
print(ner_tags)
# Counter({u'O': 1146068, u'geo': 58388, u'org': 48094, u'per': 44254, u'tim': 34789, u'gpe': 20680, u'art': 867, u'eve': 709, u'nat': 300})
 
print("Words=", sum(ner_tags.values()))
# Words= 1354149

Counter({'O': 1146068, 'geo': 58388, 'org': 48094, 'per': 44254, 'tim': 34789, 'gpe': 20680, 'art': 867, 'eve': 709, 'nat': 300})
Words= 1354149


In [48]:
import string
from nltk.stem.snowball import SnowballStemmer
 
def features(tokens, index, history):
    """
    `tokens`  = a POS-tagged sentence [(w1, t1), ...]
    `index`   = the index of the token we want to extract features for
    `history` = the previous predicted IOB tags
    """
 
    # init the stemmer
    stemmer = SnowballStemmer('english')
 
    # Pad the sequence with placeholders
    tokens = [('[START2]', '[START2]'), ('[START1]', '[START1]')] + list(tokens) + [('[END1]', '[END1]'), ('[END2]', '[END2]')]
    history = ['[START2]', '[START1]'] + list(history)
 
    # shift the index with 2, to accommodate the padding
    index += 2
 
    word, pos = tokens[index]
    prevword, prevpos = tokens[index - 1]
    prevprevword, prevprevpos = tokens[index - 2]
    nextword, nextpos = tokens[index + 1]
    nextnextword, nextnextpos = tokens[index + 2]
    previob = history[index - 1]
    contains_dash = '-' in word
    contains_dot = '.' in word
    allascii = all([True for c in word if c in string.ascii_lowercase])
 
    allcaps = word == word.capitalize()
    capitalized = word[0] in string.ascii_uppercase
 
    prevallcaps = prevword == prevword.capitalize()
    prevcapitalized = prevword[0] in string.ascii_uppercase
 
    nextallcaps = prevword == prevword.capitalize()
    nextcapitalized = prevword[0] in string.ascii_uppercase
 
    return {
        'word': word,
        'lemma': stemmer.stem(word),
        'pos': pos,
        'all-ascii': allascii,
 
        'next-word': nextword,
        'next-lemma': stemmer.stem(nextword),
        'next-pos': nextpos,
 
        'next-next-word': nextnextword,
        'nextnextpos': nextnextpos,
 
        'prev-word': prevword,
        'prev-lemma': stemmer.stem(prevword),
        'prev-pos': prevpos,
 
        'prev-prev-word': prevprevword,
        'prev-prev-pos': prevprevpos,
 
        'prev-iob': previob,
 
        'contains-dash': contains_dash,
        'contains-dot': contains_dot,
 
        'all-caps': allcaps,
        'capitalized': capitalized,
 
        'prev-all-caps': prevallcaps,
        'prev-capitalized': prevcapitalized,
 
        'next-all-caps': nextallcaps,
        'next-capitalized': nextcapitalized,
    }

In [49]:
def to_conll_iob(annotated_sentence):
    """
    `annotated_sentence` = list of triplets [(w1, t1, iob1), ...]
    Transform a pseudo-IOB notation: O, PERSON, PERSON, O, O, LOCATION, O
    to proper IOB notation: O, B-PERSON, I-PERSON, O, O, B-LOCATION, O
    """
    proper_iob_tokens = []
    for idx, annotated_token in enumerate(annotated_sentence):
        tag, word, ner = annotated_token
 
        if ner != 'O':
            if idx == 0:
                ner = "B-" + ner
            elif annotated_sentence[idx - 1][2] == ner:
                ner = "I-" + ner
            else:
                ner = "B-" + ner
        proper_iob_tokens.append((tag, word, ner))
    return proper_iob_tokens
 
def read_gmb(corpus_root):
    for root, dirs, files in os.walk(corpus_root):
        for filename in files:
            if filename.endswith(".tags"):
                with open(os.path.join(root, filename), 'rb') as file_handle:
                    file_content = file_handle.read().decode('utf-8').strip()
                    annotated_sentences = file_content.split('\n\n')
                    for annotated_sentence in annotated_sentences:
                        annotated_tokens = [seq for seq in annotated_sentence.split('\n') if seq]
 
                        standard_form_tokens = []
 
                        for idx, annotated_token in enumerate(annotated_tokens):
                            annotations = annotated_token.split('\t')
                            word, tag, ner = annotations[0], annotations[1], annotations[3]
 
                            if ner != 'O':
                                ner = ner.split('-')[0]
 
                            if tag in ('LQU', 'RQU'):   # Make it NLTK compatible
                                tag = "``"
 
                            standard_form_tokens.append((word, tag, ner))
 
                        conll_tokens = to_conll_iob(standard_form_tokens)
 
                        # Make it NLTK Classifier compatible - [(w1, t1, iob1), ...] to [((w1, t1), iob1), ...]
                        # Because the classfier expects a tuple as input, first item input, second the class
                        yield [((w, t), iob) for w, t, iob in conll_tokens]
reader = read_gmb(corpus_root)

In [50]:
print(next(reader))
print('------------')
"""
[((u'Thousands', u'NNS'), u'O'), ((u'of', u'IN'), u'O'), ((u'demonstrators', u'NNS'), u'O'), ((u'have', u'VBP'), u'O'), ((u'marched', u'VBN'), u'O'), ((u'through', u'IN'), u'O'), ((u'London', u'NNP'), u'B-geo'), ((u'to', u'TO'), u'O'), ((u'protest', u'VB'), u'O'), ((u'the', u'DT'), u'O'), ((u'war', u'NN'), u'O'), ((u'in', u'IN'), u'O'), ((u'Iraq', u'NNP'), u'B-geo'), ((u'and', u'CC'), u'O'), ((u'demand', u'VB'), u'O'), ((u'the', u'DT'), u'O'), ((u'withdrawal', u'NN'), u'O'), ((u'of', u'IN'), u'O'), ((u'British', u'JJ'), u'B-gpe'), ((u'troops', u'NNS'), u'O'), ((u'from', u'IN'), u'O'), ((u'that', u'DT'), u'O'), ((u'country', u'NN'), u'O'), ((u'.', u'.'), u'O')]
------------
"""
 
print(next(reader))
print('------------')
"""
[((u'Families', u'NNS'), u'O'), ((u'of', u'IN'), u'O'), ((u'soldiers', u'NNS'), u'O'), ((u'killed', u'VBN'), u'O'), ((u'in', u'IN'), u'O'), ((u'the', u'DT'), u'O'), ((u'conflict', u'NN'), u'O'), ((u'joined', u'VBD'), u'O'), ((u'the', u'DT'), u'O'), ((u'protesters', u'NNS'), u'O'), ((u'who', u'WP'), u'O'), ((u'carried', u'VBD'), u'O'), ((u'banners', u'NNS'), u'O'), ((u'with', u'IN'), u'O'), ((u'such', u'JJ'), u'O'), ((u'slogans', u'NNS'), u'O'), ((u'as', u'IN'), u'O'), ((u'"', '``'), u'O'), ((u'Bush', u'NNP'), u'B-per'), ((u'Number', u'NN'), u'O'), ((u'One', u'CD'), u'O'), ((u'Terrorist', u'NN'), u'O'), ((u'"', '``'), u'O'), ((u'and', u'CC'), u'O'), ((u'"', '``'), u'O'), ((u'Stop', u'VB'), u'O'), ((u'the', u'DT'), u'O'), ((u'Bombings', u'NNS'), u'O'), ((u'.', u'.'), u'O'), ((u'"', '``'), u'O')]
------------
"""
 
print(next(reader))
print('------------')
"""
[((u'They', u'PRP'), u'O'), ((u'marched', u'VBD'), u'O'), ((u'from', u'IN'), u'O'), ((u'the', u'DT'), u'O'), ((u'Houses', u'NNS'), u'O'), ((u'of', u'IN'), u'O'), ((u'Parliament', u'NN'), u'O'), ((u'to', u'TO'), u'O'), ((u'a', u'DT'), u'O'), ((u'rally', u'NN'), u'O'), ((u'in', u'IN'), u'O'), ((u'Hyde', u'NNP'), u'B-geo'), ((u'Park', u'NNP'), u'I-geo'), ((u'.', u'.'), u'O')]
------------
"""

[(('Masked', 'VBN'), 'O'), (('assailants', 'NNS'), 'O'), (('with', 'IN'), 'O'), (('grenades', 'NNS'), 'O'), (('and', 'CC'), 'O'), (('automatic', 'JJ'), 'O'), (('weapons', 'NNS'), 'O'), (('attacked', 'VBD'), 'O'), (('a', 'DT'), 'O'), (('wedding', 'VBG'), 'O'), (('party', 'NN'), 'O'), (('in', 'IN'), 'O'), (('southeastern', 'JJ'), 'O'), (('Turkey', 'NNP'), 'B-geo'), ((',', ','), 'O'), (('killing', 'VBG'), 'O'), (('45', 'CD'), 'O'), (('people', 'NNS'), 'O'), (('and', 'CC'), 'O'), (('wounding', 'VBG'), 'O'), (('at', 'IN'), 'O'), (('least', 'JJS'), 'O'), (('six', 'CD'), 'O'), (('others', 'NNS'), 'O'), (('.', '.'), 'O')]
------------
[(('Turkish', 'JJ'), 'B-gpe'), (('officials', 'NNS'), 'O'), (('said', 'VBD'), 'O'), (('the', 'DT'), 'O'), (('attack', 'NN'), 'O'), (('occurred', 'VBD'), 'O'), (('Monday', 'NNP'), 'B-tim'), (('in', 'IN'), 'O'), (('the', 'DT'), 'O'), (('village', 'NN'), 'O'), (('of', 'IN'), 'O'), (('Bilge', 'NNP'), 'B-geo'), (('about', 'IN'), 'O'), (('600', 'CD'), 'O'), (('kilomete

"\n[((u'They', u'PRP'), u'O'), ((u'marched', u'VBD'), u'O'), ((u'from', u'IN'), u'O'), ((u'the', u'DT'), u'O'), ((u'Houses', u'NNS'), u'O'), ((u'of', u'IN'), u'O'), ((u'Parliament', u'NN'), u'O'), ((u'to', u'TO'), u'O'), ((u'a', u'DT'), u'O'), ((u'rally', u'NN'), u'O'), ((u'in', u'IN'), u'O'), ((u'Hyde', u'NNP'), u'B-geo'), ((u'Park', u'NNP'), u'I-geo'), ((u'.', u'.'), u'O')]\n------------\n"

In [56]:
import pickle
from collections import Iterable
from nltk.tag import ClassifierBasedTagger
from nltk.chunk import ChunkParserI
from nltk.chunk.util import conlltags2tree
class NamedEntityChunker(ChunkParserI):
    def __init__(self, train_sents, **kwargs):
        assert isinstance(train_sents, Iterable)
 
        self.feature_detector = features
        self.tagger = ClassifierBasedTagger(
            train=train_sents,
            feature_detector=features,
            **kwargs)
 
    def parse(self, tagged_sent):
        chunks = self.tagger.tag(tagged_sent)
 
        # Transform the result from [((w1, t1), iob1), ...] 
        # to the preferred list of triplets format [(w1, t1, iob1), ...]
        iob_triplets = [(w, t, c) for ((w, t), c) in chunks]
 
        # Transform the list of triplets to nltk.Tree format
        return conlltags2tree(iob_triplets)

In [57]:
reader = read_gmb(corpus_root)
data = list(reader)
training_samples = data[:int(len(data) * 0.9)]
test_samples = data[int(len(data) * 0.9):]
 
print("#training samples = %s" % len(training_samples))    # training samples = 55809
print("#test samples = %s" % len(test_samples))                # test samples = 6201

#training samples = 55809
#test samples = 6201


In [58]:
chunker = NamedEntityChunker(training_samples[:2000])

In [62]:
from nltk import pos_tag, word_tokenize
print(chunker.parse(pos_tag(word_tokenize("I'm going to Germany this Monday."))))

(S
  (gpe I/PRP)
  'm/VBP
  going/VBG
  to/TO
  (geo Germany/NNP)
  this/DT
  (tim Monday/NNP)
  ./.)


# Gensim

In [1]:
import gensim
from gensim import corpora, models, similarities
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
from scipy import spatial
import numpy
def avg_feature_vector(words, model, num_features): ##implementation from https://stackoverflow.com/questions/22129943/how-to-calculate-the-sentence-similarity-using-word2vec-model-of-gensim-with-pyt
        #function to average all words vectors in a given paragraph
        featureVec = numpy.zeros((num_features,), dtype="float32")
        nwords = 0

        #list containing names of words in the vocabulary
        #index2word_set = set(model.index2word) this is moved as input param for performance reasons
        for word in words:
            if word in model.wv.index2word:
                nwords = nwords+1
                featureVec = numpy.add(featureVec, model.wv[word])
        if(nwords>0):
            featureVec = numpy.divide(featureVec, nwords)
        
        return featureVec

In [ ]:
sentence_1 = "king john is fat"
sentence_1_avg_vector = avg_feature_vector(sentence_1.split(),model =  models.Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True), num_features=300)

#get average vector for sentence 2
sentence_2 = "the queen eats a lot"
sentence_2_avg_vector = avg_feature_vector(sentence_2.split(), model =  models.Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True), num_features=300)
sens = [sentence_1, sentence_2]

if not sentence_1_avg_vector.all() * sentence_2_avg_vector.all() == 0:
    sen1_sen2_similarity =  1 - spatial.distance.cosine(sentence_1_avg_vector,sentence_2_avg_vector)
else:
    sen1_sen2_similarity = 1

In [ ]:
print("similarity: ")
print(sen1_sen2_similarity)

In [ ]:
import gensim
from gensim import corpora, models, similarities
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
from scipy import spatial
import numpy
def avg_feature_vector(words, model, num_features): ##implementation from https://stackoverflow.com/questions/22129943/how-to-calculate-the-sentence-similarity-using-word2vec-model-of-gensim-with-pyt
        #function to average all words vectors in a given paragraph
        featureVec = numpy.zeros((num_features,), dtype="float32")
        #list containing names of words in the vocabulary
        #index2word_set = set(model.index2word) this is moved as input param for performance reasons
        featureVec = model[words]
        return featureVec
sentence_1 = "king"
sentence_1_avg_vector = avg_feature_vector(sentence_1,model = gensim.models.Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True), num_features=300)

#get average vector for sentence 2
sentence_2 = "queen"
sentence_2_avg_vector = avg_feature_vector(sentence_2, model = gensim.models.Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True), num_features=300)
sens = [sentence_1, sentence_2]

if not sentence_1_avg_vector.all() * sentence_2_avg_vector.all() == 0:
    sen1_sen2_similarity =  1 - spatial.distance.cosine(sentence_1_avg_vector,sentence_2_avg_vector)
else:
    sen1_sen2_similarity = 1
print("similarity: ")
print(sen1_sen2_similarity)